<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Setup Script Language Container with GPU Support

We will create a custom Script Language Container with GPU support and install two machine learning libraries to it:
* `pytorch`
* `exasol-transformers-extension`

## Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Check that we don't have a SaaS backend configured

In [ ]:
from exasol.nb_connector.ai_lab_config import StorageBackend, AILabConfig
if ai_lab_config.get(AILabConfig.storage_backend, ) == StorageBackend.saas.name:
    popup_message(f"This tutorial will not work correctly with a SaaS as backend. You can export the Script-Languages-Container to a local file, but the upload to the database will fail.")

## Preparation
Similar to the [Script Languages Container tutorial](../script_languages_container/using_the_script_languages_container_tool.ipynb), we instantiate the class `ScriptLanguagesContainer`. Exasol provides a special flavor for CUDA enabled Script Language Container: `template-Exasol-8-python-3.10-cuda-conda`. This flavor already brings basic packages (e.g. CUDA) and configurations which are necessary to use GPU's in UDF's.


In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer
flavor = "template-Exasol-8-python-3.10-cuda-conda"
slc = ScriptLanguageContainer.create(secrets=ai_lab_config, name="gpu_slc", flavor=flavor)
print(f"{slc.name} is ready.")

### Add ML Libraries
In this example we use the [pytorch](https://anaconda.org/conda-forge/pytorch) Conda package to create a basic UDF and check if the GPU is available. We specify explictily to use the CUDA 12.9 version of `pytorch`, the same CUDA version which is already installed in `template-Exasol-8-python-3.10-cuda-conda`. Also, we add Exasol's `exasol-transformers-extension` package which will be used in the more advanced example.

In [ ]:
from exasol.nb_connector.slc import CondaPackageDefinition, PipPackageDefinition
pytorch_pkg = CondaPackageDefinition(pkg="pytorch", version="2.7.1=cuda129_mkl_py310_hecdad2d_304")
slc.append_custom_conda_packages([pytorch_pkg])
te_pkg = PipPackageDefinition(pkg="exasol-transformers-extension", version="2.2.1")
test_pkg = PipPackageDefinition(pkg="#test", version="2.2.1")
slc.append_custom_pip_packages([te_pkg, test_pkg])


### Export the Container to a File
We can now build and export the new Script Language Container to a tar.gz file. As the file gets quite large, we do not want to copy the file to an export path. The internal file is enough for this tutorial. The class `ScriptLanguageContainer` offers the method `export_no_copy` for this use case. **Note:** this step might take very long (>1h).

In [ ]:
slc.export_no_copy()
print("Ready")

### Deploy the Container to the Database
Now we are ready to deploy the new Script Language Container to the database. **Note:** this step might also take very long...

In [ ]:
slc.deploy()
print("Ready")

### Finish
Now you can continue with [Basic UDF with GPU support](./basic_udf_with_gpu.ipynb)